# Language Learning

## Imports

In [114]:
from docx import Document
import translators as ts
from typing import List, Tuple

import pyttsx3


## Constants

In [99]:
file = "Files/hpmor.docx"
saving_directory = "Files"

## Helpers

In [42]:
# FROM https://stackoverflow.com/a/31505798/16185542

# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    
    if sentences == []:
        sentences = [text.strip()]
    return sentences

## Get Text

In [90]:
# Open the Word document
document = Document(file)

# Extract the text
input_text = []
for para in document.paragraphs[:20]:
    sentences = split_into_sentences(para.text)
    input_text.append(sentences)

In [91]:
# Test the rest of this notebook only using the first 100 paragraphs
input_text = input_text[:100]

## Get translation

In [96]:
def translate(input_text: List[List[str]], from_lang: str, to_lang: str) -> List[Tuple[str, str]]:
    """Translate a list of paragraphs, each of which is a list of sentences, from the from_lang language to the to_lang language.

    Args:
        input_text (List[List[str]]): A list of paragraphs, each of which is a list of sentences, all in the from_lang language.
        from_lang (str): The language of the input_text. 
        to_lang (str): The language to translate the input_text to.

    Returns:
        List[Tuple[str, str]]: A list of tuples of the form (input_sentence, output_sentence), where input_sentence is a sentence in the from_lang language and output_sentence is the translation of input_sentence to the to_lang language.
    """
    output_sentence_tuples = []
    for paragraph in input_text:
        for sentence in paragraph:
            output_sentence = ts.translate_text(sentence, to_language=to_lang, from_language=from_lang, engine="deepl")
            output_sentence_tuples.append((sentence, output_sentence))
    return output_sentence_tuples

output_text = translate(input_text, "de", "en")

[('Harry Potter and the Methods of Rationality - Deutsche Fan-Übersetzung', 'Harry Potter and the Methods of Rationality - Deutsche Fan-Übersetzung'), ('Eliezer Yudkowsky', 'Eliezer Yudkowsky'), ('Vorwort von Eliezer Yudkowsky', 'Foreword by Eliezer Yudkowsky'), ('Der Text enthält viele Hinweise: offensichtliche Hinweise, nicht so offensichtliche Hinweise, wirklich obskure Hinweise, von denen ich schockiert war, dass sie von einigen Lesern erfolgreich entschlüsselt wurden, und massive Hinweise, die einfach liegen gelassen wurden.', 'The text contains many clues: obvious clues, not-so-obvious clues, really obscure clues that I was shocked were successfully deciphered by some readers, and massive clues that were simply left behind.'), ('Dies ist eine rationalistische Geschichte; ihre Rätsel sind lösbar und dazu gedacht, gelöst zu werden.', 'This is a rationalist story; their puzzles are solvable and meant to be solved.'), ('Das Tempo der Geschichte ist das einer seriellen Fiktion, d. h. 

## Generate audio

In [128]:
class Audio:
    def __init__(self, voice: int=0, speed: int=200) -> None:
        self.voice = voice
        self.speed = speed

        self.engine = self.set_engine()

    def set_engine(self):
        engine = pyttsx3.init()
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[self.voice].id)
        engine.setProperty('rate', self.speed)
        return engine

    def save_audio(self, string: str="", save_path: str="."):
        self.engine.save_to_file(string, save_path)
        self.engine.runAndWait()
        self.engine.stop()

In [133]:
german_engine = Audio(voice=2, speed=150)
english_engine = Audio(voice=0, speed=200)
for (i, (de_sentence, en_sentence)) in enumerate(output_text):
    german_engine.save_audio(de_sentence, f"{saving_directory}/de_{i}.mp3")
    english_engine.save_audio(en_sentence, f"{saving_directory}/en_{i}.mp3")

In [135]:
import glob
from pydub import AudioSegment

In [142]:
from pydub import AudioSegment
from pydub.utils import which

AudioSegment.converter = which("ffmpeg")

In [146]:
import wave

def combine_audio(file1, file2, output_file):
    with wave.open(file1, 'rb') as wav1, wave.open(file2, 'rb') as wav2, wave.open(output_file, 'wb') as wav_out:
        # Set the parameters for the output file
        wav_out.setparams(wav1.getparams())

        # Write the frames from the first file
        wav_out.writeframes(wav1.readframes(wav1.getnframes()))

        # Write the frames from the second file
        wav_out.writeframes(wav2.readframes(wav2.getnframes()))

combine_audio(f"{saving_directory}/English/file0.mp3", f"{saving_directory}/German/file0.mp3", f"{saving_directory}/file0.mp3")

In [141]:
from pydub import AudioSegment

def combine_mp3(file1, file2, output_file):
    sound1 = AudioSegment.from_file(file1, format="mp3")
    sound2 = AudioSegment.from_file(file2, format="mp3")
    combined_sound = sound1 + sound2
    combined_sound.export(output_file, format="mp3")

combine_mp3(f"{saving_directory}/English/file0.mp3", f"{saving_directory}/German/file0.mp3", f"{saving_directory}/file0.mp3")

CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 2023-01-04-git-4a80db5fc2-essentials_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.1.0 (Rev2, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libvpl --enable-libgme --en

In [140]:
# Combine the audio files into one file in the order: German-file1, English-file1, German-file2, English-file2, etc.
english_files = sorted(glob.glob(f"{saving_directory}/English/*.mp3"), key=lambda x: int(x.split("/")[-1].split(".")[0].split("file")[-1]))
german_files = sorted(glob.glob(f"{saving_directory}/German/*.mp3"), key=lambda x: int(x.split("/")[-1].split(".")[0].split("file")[-1]))

combined_audio = AudioSegment.empty()
for (english_file, german_file) in zip(english_files, german_files):
    combined_audio += AudioSegment.from_mp3(german_file)
    combined_audio += AudioSegment.from_mp3(english_file)
    
combined_audio.export(f"{saving_directory}/combined_audio.mp3", format="mp3")


CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 2023-01-04-git-4a80db5fc2-essentials_build-www.gyan.dev Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.1.0 (Rev2, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-ffnvcodec --enable-nvdec --enable-nvenc --enable-d3d11va --enable-dxva2 --enable-libvpl --enable-libgme --en

## Play audio